Multi-variate pattern analysis (with cross-validation) performed on masked brain images as part of my undergraduate neuroscience thesis/dissertation.

Written 2021.

In [ ]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.data import Brain_Data
from nltools.mask import expand_mask
from bids import BIDSLayout, BIDSValidator
from nilearn.plotting import view_img_on_surf
from bids import BIDSLayout, BIDSValidator
import nibabel as nib
import itertools as it 
from tqdm import tqdm
from nltools.analysis import Roc

def summary(self):
        """ Display a formatted summary of ROC analysis. """

        print("------------------------")
        print(".:ROC Analysis Summary:.")
        print("------------------------")
        print("{:20s}".format("Accuracy:") + "{:.4f}".format(self.accuracy))
        print("{:20s}".format("Accuracy SE:") + "{:.4f}".format(self.accuracy_se))
        print("{:20s}".format("Accuracy p-value:") + "{:.4f}".format(self.accuracy_p))
        print("{:20s}".format("Sensitivity:") + "{:.4f}".format(self.sensitivity))
        print("{:20s}".format("Specificity:") + "{:.4f}".format(self.specificity))
        print("{:20s}".format("AUC:") + "{:.4f}".format(self.auc))
        print("{:20s}".format("PPV:") + "{:.4f}".format(self.ppv))
        print("------------------------")
        
beta_dir = '/filepath/work/betas_mni/'
data_dir = '/filepath/restruc/'
layout = BIDSLayout(data_dir, derivatives=False)

In [ ]:
#CONVERT TO GRP/MNI TEMPLATE BEFORE ANY OF THE FOLLOWING

from bids import BIDSLayout, BIDSValidator
import nibabel as nib

tr = 2.0

# Set path to where the data are located
data_dir = '/filepath/restruc/'
save_dir = '/different-filepath/matt/'

# Create PyBIDS Layout
layout = BIDSLayout(data_dir, derivatives=False, validate=False)

sub_list = layout.get_subjects(task = 'auditoryperception')
sub_list = [x for x in sub_list if x not in ['05', '11', '20']] # Drop S05, S11, and S20 because they are missing too many runs
# Note: A portion of the line of code above was obtained from https://stackoverflow.com/questions/10406130/check-if-something-is-not-in-a-list-in-python

exclusions = {'01':[], '02':[2], '03':[], '04':[], '06':[], '07':[], '08':[6,7,8], '09':[], '10':[5,6,7], '12':[], '13':[7,8], '14':[], '15':[], '16':[], '17':[], '18':[6], '19':[3,4]}

print(sub_list)

In [ ]:
# Load mask
mask_dir = glob.glob(os.path.join(data_dir, 'templates', 'grpbold7Tp1', 'mnimask*'))[0]
mask = Brain_Data(mask_dir)
mask_x = expand_mask(mask)

# Plot mask
mask.plot()

In [ ]:
run_list

In [ ]:
# FULL MASK MVPA

# List genres
genre_list = ['ambient', 'country', 'metal', 'rocknroll', 'symphonic']

# Define all genre pairs
combinations = list(it.combinations(genre_list, 2))

cv_results = []
results = []

# Create a loop to create a classifier to distinguish each possible pair of genres
for mask in tqdm(mask_dir):
    
    mask = Brain_Data(mask)
    mask_x = expand_mask(mask)
    
    for sub in sub_list:
        
        run_list = layout.get_runs(subject=sub)
        numruns =len(run_list)

        # Locate, sort, append, and read the files for the first genre in the pair as a Brain Data object
        genre1_file_list = glob.glob(os.path.join(beta_dir, f'sub-{sub}*ambient*.nii.gz'))
        genre1_file_list.sort()
        genre1 = Brain_Data(genre1_file_list)

        # Locate, sort, append, and read the files for the second genre in the pair as a Brain Data object
        genre2_file_list = glob.glob(os.path.join(beta_dir, f'sub-{sub}*country*.nii.gz'))
        genre2_file_list.sort()
        genre2 = Brain_Data(genre2_file_list)

        # Append data for both genres' Brain Data objects into one object
        data = genre1.append(genre2)

        # Mask the data
        data = data.apply_mask(mask)

        # Create a set of labels to separate the two genres in the pair being classified
        Y = pd.DataFrame(np.hstack([np.ones(len(genre1_file_list)), np.zeros(len(genre2_file_list))]))

        # Assign the set of labels to the Brain Data object
        data.Y = Y

        # Pass a vector indicating subject labels as grouping variable
        run_list_x = [os.path.basename(x).split('_')[1] for x in genre2_file_list]
        run_id = pd.DataFrame(run_list_x + run_list_x)

        # Run leave-one-subject-out cross-validated suport vector machine model; separate subjects by grouping them together;
        svm_stats_masked = data.predict(algorithm='svm', cv_dict={'type': 'kfolds','n_folds':numruns, 'subject_id':run_id}, **{'kernel':"linear"})

        # Write stats file to 'Outputs' directory
        #svm_stats_masked['weight_map'].write(f'Outputs/svm_stats/{pair[0]}_vs_{pair[1]}_svm_stats_fMRI_bold.nii.gz')

        roc = Roc(input_values=svm_stats_masked['dist_from_hyperplane_xval'],
                  binary_outcome=svm_stats_masked['Y'].astype(bool))
        roc.plot()
        roc.summary()

        print("{:20s}".format("Accuracy:") + "{:.4f}".format(roc.accuracy))
        print("{:20s}".format("Accuracy SE:") + "{:.4f}".format(roc.accuracy_se))
        print("{:20s}".format("Accuracy p-value:") + "{:.4f}".format(roc.accuracy_p))
        print("{:20s}".format("Sensitivity:") + "{:.4f}".format(roc.sensitivity))
        print("{:20s}".format("Specificity:") + "{:.4f}".format(roc.specificity))
        
        cv_results.append(svm_stats_masked['mcr_xval'])
        results.append(roc.accuracy)
    # Perhaps try stratified sampling later

In [ ]:
np.savetxt('cvaccs.csv', cv_results, delimiter=',')
np.savetxt('accs.csv', results, delimiter=',')